## Import libraries and load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pickle

%matplotlib inline

In [35]:
# Read Data

filepath = '/Users/kevinzhao/Kaggle/Recruit_Restaurant/inputs/'

air_reserve = pd.read_csv(filepath + 'air_reserve.csv')
hpg_reserve = pd.read_csv(filepath + 'hpg_reserve.csv')
air_store_info = pd.read_csv(filepath + 'air_store_info.csv')
hpg_store_info = pd.read_csv(filepath + 'hpg_store_info.csv')
air_visit_data = pd.read_csv(filepath + 'air_visit_data.csv')

store_id_relation = pd.read_csv(filepath + 'store_id_relation.csv')

## Merge dataframes

In [5]:
# Join Store reservation and store info dataframes 

air_data = air_reserve.merge(air_store_info, on='air_store_id', how='left')
hpg_data = hpg_reserve.merge(hpg_store_info, on='hpg_store_id', how='left')

In [6]:
# Add hpg ids to air data and air ids to hpg data

air_data = air_data.merge(store_id_relation, on ='air_store_id', how='left')
hpg_data = hpg_data.merge(store_id_relation, on ='hpg_store_id', how='left')

In [7]:
# Merge data sets 

# air_data_full = air_data.merge(hpg_data, on =['hpg_store_id', 'reserve_datetime', 'visit_datetime'], how='left')
# hpg_data_full = hpg_data.merge(air_data, on = ['air_store_id', 'reserve_datetime', 'visit_datetime'], how='left')
air_data_full = air_data.merge(hpg_data, on =['air_store_id', 'reserve_datetime', 'visit_datetime'], how='outer')
hpg_data_full = hpg_data.merge(air_data, on = ['hpg_store_id', 'reserve_datetime', 'visit_datetime'], how='outer')
matched_data = hpg_data_full.dropna()

In [23]:
air_data_full.shape, hpg_data_full.shape

((2088053, 15), (2088053, 15))

In [59]:
len(matched_data)

2143

In [61]:
# 87 Rows where reservation info differs. What is going on here?

matched_data[matched_data.reserve_visitors_x != matched_data.reserve_visitors_y]

,hpg_store_id_x,visit_datetime,reserve_datetime,reserve_visitors_x,hpg_genre_name,hpg_area_name,latitude_x,longitude_x,air_store_id,reserve_visitors_y,air_genre_name,air_area_name,latitude_y,longitude_y,hpg_store_id_y
24003,hpg_ab64d4038f396dc3,2016-01-16 18:00:00,2016-01-14 18:00:00,2,Japanese style,Fukuoka-ken Kitakyūshū-shi Aoyama,33.861840,130.751425,air_622375b4815cf5cb,8.0,Izakaya,Fukuoka-ken Kitakyūshū-shi None,33.866465,130.764923,hpg_ab64d4038f396dc3
314443,hpg_ac318be70910278f,2016-04-27 18:00:00,2016-04-11 14:00:00,2,International cuisine,Ōsaka-fu Ōsaka-shi Awajimachi,34.686482,135.504174,air_32460819c7600037,4.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_ac318be70910278f
904834,hpg_e6594f29f40fe2aa,2016-10-27 18:00:00,2016-10-25 08:00:00,5,International cuisine,Fukuoka-ken Fukuoka-shi None,33.556881,130.415134,air_eec5e572b9eb9c23,6.0,Izakaya,Fukuoka-ken Fukuoka-shi Shiobaru,33.561569,130.426428,hpg_e6594f29f40fe2aa
918938,hpg_832ba309e6699258,2016-10-29 19:00:00,2016-10-17 18:00:00,17,Italian,Tōkyō-to Hachiōji-shi Ishikawamachi,35.677207,139.373870,air_48f4da6223571da4,19.0,Italian/French,Tōkyō-to Tachikawa-shi Izumichō,35.714014,139.407843,hpg_832ba309e6699258
919554,hpg_c34b496d0305a809,2016-10-29 19:00:00,2016-10-28 18:00:00,4,Italian,Shizuoka-ken Hamamatsu-shi Sukenobuchō,34.721644,137.738944,air_a24bf50c3e90d583,3.0,Dining bar,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,hpg_c34b496d0305a809
920565,hpg_878cc70b1abc76f7,2016-10-29 20:00:00,2016-10-11 00:00:00,12,Seafood,Hokkaidō Asahikawa-shi 3 Jōdōri,43.768033,142.359664,air_db80363d35f10926,13.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7
920566,hpg_878cc70b1abc76f7,2016-10-29 20:00:00,2016-10-11 00:00:00,20,Seafood,Hokkaidō Asahikawa-shi 3 Jōdōri,43.768033,142.359664,air_db80363d35f10926,13.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7
930669,hpg_1803b7bed3a5927e,2016-11-02 19:00:00,2016-10-24 12:00:00,3,Japanese style,None None None,43.071045,141.309261,air_04cae7c1bc9b2a0b,2.0,Izakaya,Hokkaidō Sapporo-shi Kotoni 2 Jō,43.074470,141.300889,hpg_1803b7bed3a5927e
939955,hpg_e63863c8987bd064,2016-11-04 19:00:00,2016-11-01 17:00:00,3,Japanese style,Kanagawa-ken Yokohama-shi Nagatsutachō,35.512762,139.495733,air_640cf4835f0d9ba3,4.0,Izakaya,Tōkyō-to Ōta-ku Kamata,35.561257,139.716051,hpg_e63863c8987bd064
939956,hpg_e63863c8987bd064,2016-11-04 19:00:00,2016-11-01 17:00:00,3,Japanese style,Kanagawa-ken Yokohama-shi Nagatsutachō,35.512762,139.495733,air_640cf4835f0d9ba3,15.0,Izakaya,Tōkyō-to Ōta-ku Kamata,35.561257,139.716051,hpg_e63863c8987bd064


Not all air store has a hpg store and vice versa...must combine both dataframes after imputation

## Impute missing values

In [18]:
#function to impute missing values from air to hpg
def fill_air_hpg(data, feature, is_string=False):
    """
    Fills missing values from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    :param is_string: True if feature is string, false if feature is float
    """
    
    feature_x = feature + '_x'
    feature_y = feature + '_y'
    
    #if feature type is string like hpg_store_id, check for NaNs with type
    if is_string == True:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if type(row[feature_x])!=str else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if type(row[feature_y])!=str else row[feature_y], axis=1)
    #if feature type is float, check for NaNs using math.isnan()
    else:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if math.isnan(row[feature_x])==True else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if math.isnan(row[feature_y])==True else row[feature_y], axis=1)

In [50]:
#function to impute more general missing values from air to hpg
def fill_general(data, feature):
    """
    Fills missing genre and area name from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    """
    
    air_feature = 'air_'+feature
    hpg_feature = 'hpg_'+feature
    
    data[air_feature] = data.apply(
        lambda row: row[hpg_feature] if type(row[air_feature])!=str else row[air_feature], axis=1)
    data[hpg_feature] = data.apply(
        lambda row: row[air_feature] if type(row[hpg_feature])!=str else row[hpg_feature], axis=1)

Assume missing reserve visitors values for Air and HPG are the same for now

In [51]:
for feature in ['reserve_visitors', 'latitude', 'longitude', 'hpg_store_id']:
    if feature == 'hpg_store_id':
        fill_air_hpg(air_data_full, feature, is_string=True)
    else:
        fill_air_hpg(air_data_full, feature, is_string=False)

In [52]:
for feature in ['genre_name', 'area_name']:
    fill_general(air_data_full, feature)

In [22]:
air_data_full.longitude_y.isnull().value_counts()

True     1126438
False     961615
Name: longitude_y, dtype: int64

In [30]:
air_data_full.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,hpg_bcecaa70222d94e5,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,hpg_bcecaa70222d94e5,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,hpg_878cc70b1abc76f7,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819


## Clean up features

Reformat from string to datetime

In [35]:
from datetime import datetime
from dateutil.parser import parse

In [48]:
air_data_full.loc[:,'visit_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['visit_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)
air_data_full.loc[:,'reserve_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['reserve_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)

Remove and rename duplicate columns
- latitude, longitude, hpg_store_id, hpg_genre_name, hpg_area_name

In [61]:
air_data_full.drop(['latitude_y', 'longitude_y', 'hpg_store_id_y', 'hpg_genre_name', 'hpg_area_name'], axis=1, inplace=True)
air_data_full.rename(columns={'latitude_x':'latitude', 'longitude_x':'longitude', 'hpg_store_id_x':'hpg_store_id',
                     'air_genre_name':'genre_name', 'air_area_name':'area_name'}, inplace=True)

In [62]:
air_data_full.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,genre_name,area_name,latitude,longitude,hpg_store_id,reserve_visitors_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1.0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,3.0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,6.0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,2.0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,5.0


## Feature Engineering

#### Date and Time

In [75]:
air_data_full.loc[:,'visit_year'] = air_data_full.apply(
    lambda row: row['visit_datetime'].year, axis=1)
air_data_full.loc[:,'visit_month'] = air_data_full.apply(
    lambda row: row['visit_datetime'].month, axis=1)
air_data_full.loc[:,'visit_day'] = air_data_full.apply(
    lambda row: row['visit_datetime'].day, axis=1)
air_data_full.loc[:,'visit_time'] = air_data_full.apply(
    lambda row: row['visit_datetime'].hour, axis=1)

In [76]:
air_data_full.loc[:,'reserve_year'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].year, axis=1)
air_data_full.loc[:,'reserve_month'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].month, axis=1)
air_data_full.loc[:,'reserve_day'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].day, axis=1)
air_data_full.loc[:,'reserve_time'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].hour, axis=1)

In [74]:
air_data_test.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,genre_name,area_name,latitude,longitude,hpg_store_id,reserve_visitors_y,date,visit_year,visit_month,visit_day,visit_time,reserve_year,reserve_month,reserve_day,reserve_time
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1.0,2016-01-01 19:00:00,2016,1,1,19,2016,1,1,16
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,3.0,2016-01-01 19:00:00,2016,1,1,19,2016,1,1,19
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,6.0,2016-01-01 19:00:00,2016,1,1,19,2016,1,1,19
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,2.0,2016-01-01 20:00:00,2016,1,1,20,2016,1,1,16
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,5.0,2016-01-01 20:00:00,2016,1,1,20,2016,1,1,1


In [77]:
# pickle dataframe to load faster
# pickle.dump(air_data_full, open('air_data_full.p', 'wb'))
air_data_full = pickle.load(open('air_data_full.p', 'rb'))

#### Genre Name

In [27]:
#add feature for restaurants with Japanese in the name
air_data_full.loc[:,'Japanese_explicit'] = air_data_full.apply(
    lambda row: 1 if 'Japanese' in str(row.genre_name) else 0, axis=1)

In [28]:
#add feature for restaurants with Japanese food
air_data_full.loc[:,'Japanese'] = air_data_full.apply(
    lambda row: 1 if ('Japanese' in str(row.genre_name) or 'Izakaya' in str(row.genre_name)
                      or 'Sukiyaki' in str(row.genre_name) or 'Teppanyaki' in str(row.genre_name)
                      or 'Sushi' in str(row.genre_name)) 
                else 0, axis=1)

In [24]:
#add feature for Italian restaurants
air_data_full.loc[:,'Italian'] = air_data_full.apply(
    lambda row: 1 if ('Italian' in str(row.genre_name) or 'Pizza' in str(row.genre_name)) else 0, axis=1)

In [29]:
#add feature for Korean restaurants
air_data_full.loc[:,'Korean'] = air_data_full.apply(
    lambda row: 1 if 'Korean' in str(row.genre_name) else 0, axis=1)

In [30]:
#add feature for restaurants with Chinese food
air_data_full.loc[:,'Chinese'] = air_data_full.apply(
    lambda row: 1 if ('Chinese' in str(row.genre_name) or 'Cantonese' in str(row.genre_name)
                      or 'Shanghai' in str(row.genre_name) or 'Sichuan' in str(row.genre_name)
                      or 'Dim Sum' in str(row.genre_name) or 'Taiwanese' in str(row.genre_name)) 
                else 0, axis=1)

In [32]:
pickle.dump(air_data_full, open('air_data_full2.p', 'wb'))
air_data_full = pickle.load(open('air_data_full2.p', 'rb'))

#### Area Name

- Prefecture
- City